# Imports

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gymnasium as gym
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
import stable_baselines3
import gym_trading_env
from gym_trading_env.renderer import Renderer   
from pathlib import Path
from BitcoinDownloader import download_exchange_data, get_dataframes
#from BitcoinIndicators import Indicators
from ta_indicators import Indicators
from BitcoinRewards import reward_function
from tqdm import tqdm

import torch
torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

# Download Data if needed

In [2]:
download_again = False
data_path = './data/binance-BTCUSDT-1h.pkl'
if not Path(data_path).is_file() or download_again:
    download_exchange_data()
else:
    print("data already downloaded")


download_again = False
data_path = './data/bitfinex2-BTCUSDT-1h.pkl'
if not Path(data_path).is_file() or download_again:
    download_exchange_data(data_path=['bitfinex2'])
else:
    print("data already downloaded")


download_again = False
data_path = './data/huobi-BTCUSDT-1h.pkl'
if not Path(data_path).is_file() or download_again:
    download_exchange_data(data_path=['huobi'])
else:
    print("data already downloaded")

data already downloaded
data already downloaded
data already downloaded


In [3]:
bin_train_df, bin_eval_df = get_dataframes(data_path='./data/binance-BTCUSDT-1h.pkl')
bitf_train_df, bitf_eval_df = get_dataframes(data_path='./data/bitfinex2-BTCUSDT-1h.pkl')
huobi_train_df, huobi_eval_df = get_dataframes(data_path='./data/huobi-BTCUSDT-1h.pkl')

# Create features

### Create Features for Training Sets

In [4]:
bin_path = './data/binance_indicators.csv'
bitf_path = './data/bitfinex2_indicators.csv'
huobi_path = './data/huobi_indicators.csv'

# Process training sets
bin_train_df.to_csv(bin_path)
bin_indicators = Indicators(bin_path)

bitf_train_df.to_csv(bitf_path)
bitf_indicators = Indicators(bitf_path)

huobi_train_df.to_csv(huobi_path)
huobi_indicators = Indicators(huobi_path)

"""training_df.to_csv(ind_path)
indicators = Indicators(ind_path)
indicators.to_csv(ind_path)
training_df = pd.read_csv(ind_path)
training_df["date_open"] = pd.to_datetime(training_df["date_open"])
training_df.set_index("date_open", inplace=True)

training_df.dropna(inplace=True)
print(training_df.head(3))
print(training_df.tail(3))"""

d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"DX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"feature_ADX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:254: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

'training_df.to_csv(ind_path)\nindicators = Indicators(ind_path)\nindicators.to_csv(ind_path)\ntraining_df = pd.read_csv(ind_path)\ntraining_df["date_open"] = pd.to_datetime(training_df["date_open"])\ntraining_df.set_index("date_open", inplace=True)\n\ntraining_df.dropna(inplace=True)\nprint(training_df.head(3))\nprint(training_df.tail(3))'

### Create Features for Eval Sets

In [5]:
bin_eval_path = './data/binance_eval_indicators.csv'
bitf_eval_path = './data/bitfinex2_eval_indicators.csv'
huobi_eval_path = './data/huobi_eval_indicators.csv'

# Process eval sets
bin_eval_df.to_csv(bin_eval_path)
bin_eval_indicators = Indicators(bin_eval_path)

"""bitf_eval_df.to_csv(bitf_path)
bitf_eval_indicators = Indicators(bitf_eval_path)

huobi_eval_df.to_csv(huobi_path)
huobi_eval_indicators = Indicators(huobi_eval_path)"""

d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"DX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"feature_ADX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:254: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

'bitf_eval_df.to_csv(bitf_path)\nbitf_eval_indicators = Indicators(bitf_eval_path)\n\nhuobi_eval_df.to_csv(huobi_path)\nhuobi_eval_indicators = Indicators(huobi_eval_path)'

### Write all datasets to pkl files

Training environment will use directory of all three pkl files.

In [6]:
bin_indicators.to_pickle('./data/train/binance.pkl')
bitf_indicators.to_pickle('./data/train/bitfinex.pkl')
huobi_indicators.to_pickle('./data/train/huobi.pkl')


bin_eval_indicators.to_pickle('./data/eval/binance.pkl')
#bitf_eval_indicators.to_pickle('./data/eval/bitfinex.pkl')
#huobi_eval_indicators.to_pickle('./data/eval/huobi.pkl')


d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.indicators.dropna(inplace=True)


### Create single dataframe for eval dataset

Eval environment will just use a single dataframe

In [7]:
eval_df = pd.read_pickle('./data/eval/binance.pkl')

eval_df.tail()

,close,open,high,low,volume,feature_ROC_2,feature_ROC_4,feature_ROC_6,feature_ROC_8,feature_ROC_16,...,feature_RSI_16,feature_RSI_32,feature_RSI_64,feature_ATR_2,feature_ATR_4,feature_ATR_6,feature_ATR_8,feature_ATR_16,feature_ATR_32,feature_ATR_64
date_open,,,,,,,,,,,,,,,,,,,,,
2023-08-07 13:00:00,29030.51,29079.25,29089.90,29030.00,1267.88152,-0.222227,-0.132988,-0.125848,-0.373479,-0.067575,...,31.362001,31.773049,33.000448,87.560,74.7075,64.611667,65.38750,71.693750,63.398437,50.562813
2023-08-07 14:00:00,29023.99,29030.50,29074.56,28964.65,1714.95192,-0.194129,-0.366265,-0.129455,-0.261964,-0.147210,...,30.602412,32.325329,32.900929,84.905,85.0200,76.580000,69.52000,72.248750,64.972187,51.676563
2023-08-07 15:00:00,28858.44,29024.00,29027.07,28788.73,3624.59818,-0.576550,-0.744583,-0.556023,-0.566422,-0.373065,...,26.246603,28.595441,30.113144,174.125,130.8425,107.846667,91.99000,85.380625,71.186562,54.249375
2023-08-07 16:00:00,28938.00,28858.45,28976.04,28701.03,3745.28186,-0.320760,-0.608690,-0.702484,-0.542351,-0.425681,...,30.078697,30.048021,31.181249,256.675,170.7900,142.238333,121.60375,97.709375,78.239062,57.686094
2023-08-07 17:00:00,28981.32,28937.99,28989.99,28868.00,1895.79511,0.405829,-0.289847,-0.554122,-0.492200,-0.512009,...,28.433595,31.466360,32.224288,198.500,186.3125,153.395000,130.51000,96.020000,80.855937,58.578281


# Create Environment

In [8]:
"""training_env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = training_df, # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        #trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        #borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
        #dynamic_feature_functions = [dynamic_features]
        # reward_function = reward_function,
        portfolio_initial_value = 1000,
        reward_function = reward_function
        #max_episode_duration = 1000,
    )"""


training_env = gym.make("MultiDatasetTradingEnv",
        name= "BTCUSD",
        dataset_dir= './data/train/*.pkl', # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        #trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        #borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
        #dynamic_feature_functions = [dynamic_features]
        # reward_function = reward_function,
        portfolio_initial_value = 1000,
        reward_function = reward_function
        #max_episode_duration = 1000,
)

observation, info = training_env.reset()
print(observation, info)

[ 2.4272878e-01  2.4033581e-01  3.8472018e-01  2.6634541e-01
  6.8100756e-01  5.0629693e-01  3.0035012e+00  1.0070157e+03
 -1.1807848e+03  1.8093023e+01  8.2216513e-01  4.8368992e+01
 -1.1546900e+02  1.1348732e+03  9.4944366e+01  9.5220901e+01
  9.0921761e+01  8.3264366e+01  7.4502037e+01  6.5129539e+01
  8.2128197e+01  5.0000000e+01  4.3296089e+01  4.6428570e+01
  3.6662750e+01  3.7814728e+01  3.7822086e+01  3.6450191e+01
  7.4499998e+00  7.1250000e+00  8.5666666e+00  9.2375002e+00
  1.0162500e+01  7.6531248e+00  1.1439062e+01  1.0000000e+00
  1.0000000e+00] {'idx': 0, 'step': 0, 'date': numpy.datetime64('2019-03-20T15:00:00.000000000'), 'position_index': 1, 'position': 1, 'real_position': 1, 'data_volume': 0.07574493, 'data_open': 4021.2, 'data_high': 4021.2, 'data_close': 4021.2, 'data_low': 4021.2, 'portfolio_valuation': 1000.0, 'portfolio_distribution_asset': 0.24868198547697207, 'portfolio_distribution_fiat': 0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_

# Create Model

In [9]:
model = RecurrentPPO('MlpLstmPolicy', # feed-forward neural network with multiple hidden layers
            training_env, # environment in which the agent interacts and learns
            verbose=1, # enables the training progress to be printed during the learning process
            gamma=0.95, # determines the importance of future rewards compared to immediate rewards
            n_steps=144, # steps to collect samples from the environment before performing an update
            ent_coef=0.01, # encourages exploration by adding entropy to the policy loss
            learning_rate=0.0003, # controls the step size at which model's parameters are updated based on the gradient of the loss function
            clip_range=0.1, # limits the update to a certain range to prevent large policy updates
            device="cuda" if torch.cuda.is_available() else "cpu",
            n_epochs=10,
            )
#n = len(training_df)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


### Train Model (can take a while)

In [ ]:
model.learn(total_timesteps=300000)

In [ ]:
model.save('./models/RPPO_totalSteps300k_nsteps144_LR0.0003_epochs10')

### Load Model (if applicable)

In [13]:
model = RecurrentPPO.load('./models/RPPO_totalSteps300k_nsteps144_LR0.0003_epochs10')

Exception ignored in: <_io.FileIO name='models\\RPPO_totalSteps300k_nsteps144_LR0.0003_epochs10' mode='rb' closefd=True>
Traceback (most recent call last):
  File "d:\Python\Lib\site-packages\stable_baselines3\common\base_class.py", line 679, in load
    data, params, pytorch_variables = load_from_zip_file(
                                      ^^^^^^^^^^^^^^^^^^^


### This confirms that the previously-trained model has been loaded

ep_info_buffer should be non-null

In [14]:
model.ep_info_buffer

deque([{'r': -615094.354121, 'l': 46788, 't': 305.132658},
       {'r': -615094.32079, 'l': 46788, 't': 684.419222},
       {'r': -615094.289739, 'l': 46788, 't': 1028.594519},
       {'r': -615094.299454, 'l': 46788, 't': 1379.696704},
       {'r': -615094.265168, 'l': 46788, 't': 1708.018902},
       {'r': -615094.340706, 'l': 46788, 't': 2044.228919}],
      maxlen=100)

# Calculate indicators for testing dataset

In [18]:
'''ind_path = './data/indicators.csv'
testing_df.to_csv(ind_path)
indicators = Indicators(ind_path)
indicators.to_csv(ind_path)
testing_df = pd.read_csv(ind_path)
testing_df["date_open"] = pd.to_datetime(testing_df["date_open"])
testing_df.set_index("date_open", inplace=True)'''

eval_df.dropna(inplace=True)
eval_df.tail(3)


,close,open,high,low,volume,feature_ROC_2,feature_ROC_4,feature_ROC_6,feature_ROC_8,feature_ROC_16,...,feature_RSI_16,feature_RSI_32,feature_RSI_64,feature_ATR_2,feature_ATR_4,feature_ATR_6,feature_ATR_8,feature_ATR_16,feature_ATR_32,feature_ATR_64
date_open,,,,,,,,,,,,,,,,,,,,,
2023-08-07 15:00:00,28858.44,29024.00,29027.07,28788.73,3624.59818,-0.576550,-0.744583,-0.556023,-0.566422,-0.373065,...,26.246603,28.595441,30.113144,174.125,130.8425,107.846667,91.99000,85.380625,71.186562,54.249375
2023-08-07 16:00:00,28938.00,28858.45,28976.04,28701.03,3745.28186,-0.320760,-0.608690,-0.702484,-0.542351,-0.425681,...,30.078697,30.048021,31.181249,256.675,170.7900,142.238333,121.60375,97.709375,78.239062,57.686094
2023-08-07 17:00:00,28981.32,28937.99,28989.99,28868.00,1895.79511,0.405829,-0.289847,-0.554122,-0.492200,-0.512009,...,28.433595,31.466360,32.224288,198.500,186.3125,153.395000,130.51000,96.020000,80.855937,58.578281


In [32]:
len(eval_df)

5121

In [37]:
testing_env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = eval_df, # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        portfolio_initial_value = 1000,
        reward_function = reward_function,
    )

observation, info = testing_env.reset()
print(observation.shape)
print(observation)
print(info)

(37,)
[ 8.7677605e-02 -1.6281354e-01 -2.5204769e-01 -3.4905249e-01
 -1.1987240e-01 -3.3908841e-01  6.7657948e-01 -1.3897646e+02
  5.9759155e+01  1.0690672e+02 -3.5503836e+02 -1.1604339e+02
 -1.4387981e+02 -4.3159134e+01  9.4134003e+01  8.7169312e+01
  8.2858330e+01  8.9389130e+01  8.5556389e+01  7.5845436e+01
  6.8625328e+01  5.0000000e+01  2.2887589e+01  1.8897150e+01
  1.8681402e+01  2.9803864e+01  3.2066544e+01  3.6547573e+01
  2.1735001e+01  2.7867500e+01  2.6600000e+01  3.0045000e+01
  3.3138126e+01  3.4421562e+01  4.5555626e+01  1.0000000e+00
  1.0000000e+00]
{'idx': 0, 'step': 0, 'date': numpy.datetime64('2023-01-06T08:00:00.000000000'), 'position_index': 1, 'position': 1, 'real_position': 1, 'data_volume': 5475.1394, 'data_open': 16794.33, 'data_high': 16812.22, 'data_close': 16802.11, 'data_low': 16791.47, 'portfolio_valuation': 1000.0, 'portfolio_distribution_asset': 0.05951633455560046, 'portfolio_distribution_fiat': 0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_

# Test trained model on testing data

In [38]:
for _ in tqdm(range(len(eval_df))):
    position_index, _states = model.predict(observation)
    observation, reward, done, truncated, info = testing_env.step(position_index)
    testing_env.save_for_render(dir = "render_logs")
    if done or truncated:
        break

100%|█████████▉| 5119/5121 [04:27<00:00, 19.13it/s]

Market Return : 72.49%   |   Portfolio Return : 34.44%   |   


# Render results

In [39]:
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Aug/2023 13:24:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 13:24:49] "GET /update_data/BTCUSD_2023-08-10_13-24-39.pkl HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2023 13:24:49] "GET /metrics HTTP/1.1" 200 -
